In [42]:
import pandas as pd

# Fetch ACS data for metropolitan areas
url = (
    "https://api.census.gov/data/2021/acs/acs5?"
    "get=NAME,"
    "B25077_001E,"   # median home value
    "B19013_001E,"   # median household income
    "B23025_005E,"   # unemployed
    "B23025_003E,"   # labor force
    "B01002_001E,"   # median age
    "B25003_003E,"   # renter-occupied units
    "B25003_001E,"   # occupied units total
    "B01003_001E,"   # total population
    "B15003_001E,"   # population 25+
    "B15003_022E,"   # bachelor's
    "B15003_023E,"   # master's
    "B15003_024E,"   # professional
    "B15003_025E,"   # doctorate
    "B25002_001E,"   # total housing units
    "B25002_003E"    # vacant housing units
    "&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*"
)
df = pd.read_json(url)

df.to_csv("acs_housing_metro.csv", index=False)


In [43]:
# Preview data
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,NAME,B25077_001E,B19013_001E,B23025_005E,B23025_003E,B01002_001E,B25003_003E,B25003_001E,B01003_001E,B15003_001E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B25002_001E,B25002_003E,metropolitan statistical area/micropolitan sta...
1,"Aberdeen, SD Micro Area",168400,64194,686,23563,37.4,5862,17689,42476,28054,6428,1569,517,361,19820,2131,10100
2,"Aberdeen, WA Micro Area",213200,53615,2119,30609,44.4,8870,29319,74737,54397,5553,2705,370,249,35928,6609,10140
3,"Abilene, TX Metro Area",134000,57356,2525,80930,34.2,23739,64331,175241,111073,18184,7396,1647,894,73899,9568,10180
4,"Ada, OK Micro Area",137000,55862,776,18096,36.2,5104,14604,38102,24706,4420,2173,289,363,17362,2758,10220


In [44]:
# Change the first row to column names
df.columns = df.iloc[0]
df = df.iloc[1:].reset_index(drop=True)

# Rename columns
rename_dict = {
    "NAME": "metro_area",
    "B25077_001E": "median_home_value",
    "B19013_001E": "median_household_income",
    "B23025_005E": "unemployed",
    "B23025_003E": "labor_force",
    "B01002_001E": "median_age",
    "B25003_003E": "renter_occupied_units",
    "B25003_001E": "occupied_units_total",
    "B01003_001E": "total_population",
    "B15003_001E": "population_25_plus",
    "B15003_022E": "bachelors_degree",
    "B15003_023E": "masters_degree",
    "B15003_024E": "professional_degree",
    "B15003_025E": "doctorate_degree",
    "B25002_001E": "total_housing_units",
    "B25002_003E": "vacant_housing_units",
    "metropolitan statistical area/micropolitan statistical area": "cbsa_code"
}
df = df.rename(columns=rename_dict)

In [45]:
df.head()

,metro_area,median_home_value,median_household_income,unemployed,labor_force,median_age,renter_occupied_units,occupied_units_total,total_population,population_25_plus,bachelors_degree,masters_degree,professional_degree,doctorate_degree,total_housing_units,vacant_housing_units,cbsa_code
0,"Aberdeen, SD Micro Area",168400,64194,686,23563,37.4,5862,17689,42476,28054,6428,1569,517,361,19820,2131,10100
1,"Aberdeen, WA Micro Area",213200,53615,2119,30609,44.4,8870,29319,74737,54397,5553,2705,370,249,35928,6609,10140
2,"Abilene, TX Metro Area",134000,57356,2525,80930,34.2,23739,64331,175241,111073,18184,7396,1647,894,73899,9568,10180
3,"Ada, OK Micro Area",137000,55862,776,18096,36.2,5104,14604,38102,24706,4420,2173,289,363,17362,2758,10220
4,"Adrian, MI Micro Area",150700,60656,2513,46859,41.8,8505,38667,99346,69179,9822,4439,552,305,43604,4937,10300


In [46]:
# Convert relevant columns to numeric
cols_to_num = [
    "unemployed", "labor_force",
    "renter_occupied_units", "occupied_units_total",
    "population_25_plus",
    "bachelors_degree", "masters_degree", "professional_degree", "doctorate_degree",
    "median_home_value", "median_household_income", "median_age", "total_population",
    "vacant_housing_units", "total_housing_units"
]

df[cols_to_num] = df[cols_to_num].apply(pd.to_numeric, errors="coerce")

# Compute additional metrics
df["unemployment_rate"] = df["unemployed"] / df["labor_force"]

df["percent_renters"] = df["renter_occupied_units"] / df["occupied_units_total"]

df["percent_bachelors_plus"] = (
    df["bachelors_degree"]
    + df["masters_degree"]
    + df["professional_degree"]
    + df["doctorate_degree"]
) / df["population_25_plus"]

df["vacancy_rate"] = df["vacant_housing_units"] / df["total_housing_units"]

In [47]:
df.head()

,metro_area,median_home_value,median_household_income,unemployed,labor_force,median_age,renter_occupied_units,occupied_units_total,total_population,population_25_plus,...,masters_degree,professional_degree,doctorate_degree,total_housing_units,vacant_housing_units,cbsa_code,unemployment_rate,percent_renters,percent_bachelors_plus,vacancy_rate
0,"Aberdeen, SD Micro Area",168400,64194,686,23563,37.4,5862,17689,42476,28054,...,1569,517,361,19820,2131,10100,0.029113,0.331392,0.316354,0.107518
1,"Aberdeen, WA Micro Area",213200,53615,2119,30609,44.4,8870,29319,74737,54397,...,2705,370,249,35928,6609,10140,0.069228,0.302534,0.163189,0.183951
2,"Abilene, TX Metro Area",134000,57356,2525,80930,34.2,23739,64331,175241,111073,...,7396,1647,894,73899,9568,10180,0.031200,0.369013,0.253176,0.129474
3,"Ada, OK Micro Area",137000,55862,776,18096,36.2,5104,14604,38102,24706,...,2173,289,363,17362,2758,10220,0.042882,0.349493,0.293249,0.158853
4,"Adrian, MI Micro Area",150700,60656,2513,46859,41.8,8505,38667,99346,69179,...,4439,552,305,43604,4937,10300,0.053629,0.219955,0.218535,0.113224
